# Dataset FinanceBench

In [ ]:
# !pip install qdrant-client

In [79]:
from exp.azure_openai_conn import *
from dotenv import load_dotenv
load_dotenv()


ModuleNotFoundError: No module named 'exp'

In [73]:
import pandas as pd
import os
import requests
from datasets import load_dataset
from datasets import DatasetDict
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant


In [54]:
ds = load_dataset("PatronusAI/financebench")
df = pd.DataFrame(ds)
all_dicts = []
for index, row in df.iterrows():    
    dictionary = row['train']    
    all_dicts.append(dictionary)
df = pd.DataFrame(all_dicts)

In [ ]:
destination_folder = 'data/financebench'
# Check if the destination folder exists, and create it if it doesn't

# Check if the destination folder exists, and create it if it doesn't
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for index, row in df.iterrows():
    url = row['doc_link']
    doc_name = row['doc_name']
    
    # Add the ".pdf" extension to the doc_name
    doc_name_with_extension = doc_name + '.pdf'
    
    # Create the full file path
    file_path = os.path.join(destination_folder, doc_name_with_extension)
    
    # Send a GET request to the URL to download the file
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Save the file to the destination folder with the assigned name and ".pdf" extension
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {doc_name_with_extension}")
    else:
        print(f"Failed to download: {doc_name_with_extension} ({url})")


In [74]:
def load_chunk_persist_pdf(destination_folder) -> Qdrant:
    pdf_folder_path = destination_folder
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    chunked_documents = text_splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()

    qdrant = Qdrant.from_documents(
        chunked_documents,
        embeddings,
        path=destination_folder,
        collection_name="financebench")   

    return qdrant

In [75]:
load_chunk_persist_pdf(destination_folder)

/home/sebacastillo/.genai0/lib/python3.9/site-packages/langchain_community/embeddings/openai.py:335: UserWarning: If you have openai>=1.0.0 installed and are using Azure, please use the `AzureOpenAIEmbeddings` class.
  warnings.warn(


ValueError: Could not import qdrant-client python package. Please install it with `pip install qdrant-client`.